In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import os
import logging
import gradio as gr
import catboost
# --- 1. Setup and Data Loading ---

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(_name_)

# Global dictionary to hold all loaded data and models
APP_DATA = {}

def load_data_and_models():
    """Loads all necessary files, models, and scalers into the global APP_DATA dict."""
    logger.info("Application starting up... Loading data and models.")
    required_files = [
        'player_summary.csv', 'feature_lists.joblib', 'role_encoder.joblib',
        'scaler_bat.joblib', 'scaler_bowl.joblib', 'scaler_field.joblib',
        'batting_best_model.joblib', 'bowling_best_model.joblib', 'fielding_best_model.joblib',
        'training_metrics.csv', 'feature_importances.csv'
    ]

    missing_files = [f for f in required_files if not os.path.exists(f)]
    if missing_files:
        logger.error(f"FATAL: Missing required files: {', '.join(missing_files)}")
        return False

    try:
        df = pd.read_csv('player_summary.csv')
        df['Role'] = df['Role'].replace('wicketkeeper-batsman', 'wicketkeeper')
        APP_DATA['df'] = df[df['Recent Activity'] == True].copy()

        APP_DATA['feature_lists'] = joblib.load('feature_lists.joblib')
        APP_DATA['scalers'] = {
            'Batting': joblib.load('scaler_bat.joblib'),
            'Bowling': joblib.load('scaler_bowl.joblib'),
            'Fielding': joblib.load('scaler_field.joblib')
        }
        APP_DATA['models'] = {
            'Batting': joblib.load('batting_best_model.joblib'),
            'Bowling': joblib.load('bowling_best_model.joblib'),
            'Fielding': joblib.load('fielding_best_model.joblib')
        }
        APP_DATA['role_encoder'] = joblib.load('role_encoder.joblib')

        metrics_df = pd.read_csv('training_metrics.csv')
        APP_DATA['model_names'] = {}
        for task in ['Batting', 'Bowling', 'Fielding']:
            task_metrics = metrics_df[metrics_df['Task'] == task]
            best_model_name = task_metrics.loc[task_metrics['Test MAE'].idxmin()]['Model'].lower()
            APP_DATA['model_names'][task] = best_model_name

        logger.info("Successfully loaded all required files and models.")
        return True
    except Exception as e:
        logger.error(f"Error during startup: {str(e)}")
        return False

# --- 2. Core Application Logic ---

def predict_points_logic(player_name: str, opposition_team: str, scoring_rule: str = "default"):
    """Core logic to predict points for a single player."""
    player_stats = APP_DATA['df'][APP_DATA['df']['Player'] == player_name]
    if player_stats.empty:
        return None, f"Player '{player_name}' not found."

    player_stats, player_role = player_stats.iloc[0], player_stats.iloc[0]['Role']
    prepared_features = {}

    for task_name in ['Batting', 'Bowling', 'Fielding']:
        features = APP_DATA['feature_lists'][task_name.lower()]
        opp_cols = APP_DATA['feature_lists']['opposition']
        X_dict = {f: player_stats.get(f, 0) for f in features}
        for col in opp_cols:
            X_dict[col] = player_stats.get(col, 0) if col == f"Points vs {opposition_team}" else 0
        X_dict['Role'] = APP_DATA['role_encoder'].transform([player_role])[0]
        X_dict['Games'] = player_stats['Games']
        prepared_features[task_name] = pd.DataFrame([X_dict])

    pred_bat = max(0, APP_DATA['models']['Batting'].predict(APP_DATA['scalers']['Batting'].transform(prepared_features['Batting']))[0])
    pred_bowl = max(0, APP_DATA['models']['Bowling'].predict(APP_DATA['scalers']['Bowling'].transform(prepared_features['Bowling']))[0])
    pred_field = max(0, APP_DATA['models']['Fielding'].predict(APP_DATA['scalers']['Fielding'].transform(prepared_features['Fielding']))[0])

    if scoring_rule == 'simple_sum':
        total = pred_bat + pred_bowl + pred_field
    else: # Default scoring
        total = (pred_bat + pred_field + 4) if player_role == 'batsman' else \
                (pred_bowl + pred_field + 4) if player_role == 'bowler' else \
                (pred_bat + pred_bowl + pred_field + 4) if player_role == 'allrounder' else \
                (pred_bat + pred_field + 4) if player_role == 'wicketkeeper' else 0

    result = {
        'model_used': f"{APP_DATA['model_names']['Batting']},{APP_DATA['model_names']['Bowling']},{APP_DATA['model_names']['Fielding']}",
        'predicted_batting_points': round(pred_bat, 2),
        'predicted_bowling_points': round(pred_bowl, 2),
        'predicted_fielding_points': round(pred_field, 2),
        'predicted_total_fantasy_points': round(total, 2)
    }
    return result, None


def select_team_logic(params: dict):
    """Core logic for team selection."""
    # try:
    #     from fantasy_solver import FantasySolver
    # except ImportError:
    #     return None, None, "Critical Error: 'fantasy_solver.py' not found. This feature is unavailable."

    # Generate predictions file needed by the solver
    logger.info(f"Generating predictions for opposition team: {params['team2_name']}")
    all_preds = []
    for _, player_row in APP_DATA['df'].iterrows():
         pred_data, _ = predict_points_logic(player_row['Player'], params['team2_name'])
         if pred_data:
             all_preds.append({
                 'Player': player_row['Player'],
                 'Fantasy Points': pred_data['predicted_total_fantasy_points'],
                 'Variance': np.random.uniform(5, 25) # Placeholder for variance
             })
    pd.DataFrame(all_preds).to_csv('player_predictions.csv', index=False)

    solver = FantasySolver(player_csv='player_summary.csv', prediction_csv='player_predictions.csv')

    # Validate constraints before solving
    if params['num_team1_players'] + params['num_team2_players'] != params['total_players']:
        return None, None, "Player count mismatch: Team 1 + Team 2 players must equal Total Players."

    try:
        team_df, summary = solver.solve(params)
        if team_df is None:
            return None, None, "No feasible team found. Try adjusting budget, roles, or player counts."

        summary['format'] = params.get('format', 't20')
        return team_df, summary, None
    except Exception as e:
        logger.error(f"Error in fantasy_solver: {e}", exc_info=True)
        return None, None, f"An unexpected error occurred during solving: {e}"


# --- 3. Gradio Interface Handler Functions ---

def handle_single_prediction(player_name, opposition_team, scoring_rule):
    if not player_name or not opposition_team:
        return None, "Please select a player and an opposition team."

    scoring_rule_val = scoring_rule.lower().replace(" ", "_")
    prediction, error = predict_points_logic(player_name, opposition_team, scoring_rule_val)

    if error:
        return None, error

    df = pd.DataFrame([prediction])
    return df, None


def handle_all_predictions(opposition_team, scoring_rule):
    if not opposition_team:
        return None, "Please select an opposition team."

    scoring_rule_val = scoring_rule.lower().replace(" ", "_")

    predictions = []
    for player_name in APP_DATA['df']['Player']:
        pred, _ = predict_points_logic(player_name, opposition_team, scoring_rule_val)
        if pred:
            pred['Player'] = player_name
            pred['Role'] = APP_DATA['df'][APP_DATA['df']['Player'] == player_name]['Role'].iloc[0]
            predictions.append(pred)

    df = pd.DataFrame(predictions)
    # Reorder columns for better readability
    cols = ['Player', 'Role', 'predicted_total_fantasy_points', 'predicted_batting_points', 'predicted_bowling_points', 'predicted_fielding_points']
    return df[cols], None


def handle_team_selection(team1, team2, format_type, risk, budget, total_players, team1_players, team2_players, bat_count, bowl_count, wk_count, ar_count):
    if not team1 or not team2 or team1 == team2:
        return None, None, "Please select two different teams."

    params = {
        "total_players": int(total_players),
        "budget": int(budget),
        "role_constraints": {
            "batsman": (int(bat_count), int(bat_count)),
            "bowler": (int(bowl_count), int(bowl_count)),
            "wicketkeeper": (int(wk_count), int(wk_count)),
            "allrounder": (int(ar_count), int(ar_count))
        },
        "team1_name": team1,
        "team2_name": team2,
        "num_team1_players": int(team1_players),
        "num_team2_players": int(team2_players),
        "risk": risk.lower(),
        "format": format_type.lower()
    }

    team_df, summary, error = select_team_logic(params)

    if error:
        return None, None, error

    return team_df, summary, None


def handle_fetch_data(data_type):
    try:
        if data_type == "All Players":
            return APP_DATA['df'][['Player', 'Role', 'Games', 'Cost', 'Team']]
        elif data_type == "Training Metrics":
            return pd.read_csv("training_metrics.csv")
        elif data_type == "Feature Importances":
            return pd.read_csv("feature_importances.csv")
    except FileNotFoundError as e:
        return pd.DataFrame([{"Error": f"File not found: {e.filename}"}])
    except Exception as e:
        return pd.DataFrame([{"Error": str(e)}])

# --- 4. Gradio UI Layout ---

# Load data before building the UI. If it fails, the UI won't launch.
if load_data_and_models():
    PLAYER_LIST = sorted(APP_DATA['df']['Player'].unique().tolist())
    TEAM_LIST = sorted(APP_DATA['df']['Team'].unique().tolist())

    with gr.Blocks(theme=gr.themes.Soft(), title="Cricket Fantasy Pro") as demo:
        gr.Markdown("# 🏏 Cricket Fantasy Pro: Predict & Select")
        gr.Markdown("A standalone tool for Cricket Fantasy Points Prediction and Team Selection.")

        with gr.Tabs():
            # --- PREDICTION TAB ---
            with gr.TabItem("Player Predictions"):
                gr.Markdown("### Predict fantasy points for single or multiple players.")
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("#### Single Player Prediction")
                        player_name_input = gr.Dropdown(PLAYER_LIST, label="Select Player")
                        opposition_team_input1 = gr.Dropdown(TEAM_LIST, label="Select Opposition Team")
                        scoring_rule_input1 = gr.Radio(["Default", "Simple Sum"], label="Scoring Rule", value="Default")
                        predict_one_btn = gr.Button("Predict Player", variant="primary")

                    with gr.Column(scale=2):
                        gr.Markdown("#### Prediction Result")
                        single_pred_output = gr.DataFrame()
                        error_box1 = gr.Textbox(label="Status", interactive=False)

                gr.Markdown("---")

                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("#### All Players Prediction")
                        opposition_team_input2 = gr.Dropdown(TEAM_LIST, label="Select Opposition Team")
                        scoring_rule_input2 = gr.Radio(["Default", "Simple Sum"], label="Scoring Rule", value="Default")
                        predict_all_btn = gr.Button("Predict All Players", variant="primary")

                    with gr.Column(scale=2):
                        gr.Markdown("#### All Predictions Result")
                        all_pred_output = gr.DataFrame()
                        error_box2 = gr.Textbox(label="Status", interactive=False)

            # --- TEAM SELECTION TAB ---
            with gr.TabItem("Fantasy Team Selection"):
                gr.Markdown("### Build your optimal fantasy team based on various constraints.")
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("#### Main Constraints")
                        team1_input = gr.Dropdown(TEAM_LIST, label="Select Team 1")
                        team2_input = gr.Dropdown(TEAM_LIST, label="Select Team 2")
                        format_input = gr.Radio(["T20", "ODI", "Test"], label="Match Format", value="T20")
                        risk_input = gr.Radio(["Stable", "High-Risk", "Optimal"], label="Risk Profile", value="Stable")
                        budget_slider = gr.Slider(50, 150, value=100, step=1, label="Budget")
                        select_team_btn = gr.Button("Generate My Team", variant="primary")

                    with gr.Column(scale=1):
                        gr.Markdown("#### Player Count Constraints")
                        total_players_slider = gr.Slider(7, 15, value=11, step=1, label="Total Players in Team")
                        team1_players_slider = gr.Slider(1, 10, value=5, step=1, label="Players from Team 1")
                        team2_players_slider = gr.Slider(1, 10, value=6, step=1, label="Players from Team 2")

                        with gr.Accordion("Role Constraints (Fixed Count)", open=False):
                            bat_slider = gr.Slider(0, 8, value=4, step=1, label="Batsmen")
                            bowl_slider = gr.Slider(0, 8, value=4, step=1, label="Bowlers")
                            wk_slider = gr.Slider(0, 4, value=1, step=1, label="Wicketkeepers")
                            ar_slider = gr.Slider(0, 5, value=2, step=1, label="All-Rounders")

                gr.Markdown("---")
                gr.Markdown("### Generated Team & Summary")
                error_box3 = gr.Textbox(label="Status", interactive=False)
                with gr.Row():
                    team_output_df = gr.DataFrame(label="Optimal Team")
                    summary_output_json = gr.JSON(label="Team Summary")

            # --- DATA & METRICS TAB ---
            with gr.TabItem("Explore Data & Metrics"):
                gr.Markdown("### View raw player data, model training metrics, and feature importances.")
                with gr.Row():
                    data_type_radio = gr.Radio(["All Players", "Training Metrics", "Feature Importances"], label="Select Data to View", value="All Players")
                    fetch_data_btn = gr.Button("Fetch Data", variant="primary")

                data_output_df = gr.DataFrame()

        # --- Event Handlers ---
        predict_one_btn.click(
            fn=handle_single_prediction,
            inputs=[player_name_input, opposition_team_input1, scoring_rule_input1],
            outputs=[single_pred_output, error_box1]
        )
        predict_all_btn.click(
            fn=handle_all_predictions,
            inputs=[opposition_team_input2, scoring_rule_input2],
            outputs=[all_pred_output, error_box2]
        )
        select_team_btn.click(
            fn=handle_team_selection,
            inputs=[team1_input, team2_input, format_input, risk_input, budget_slider, total_players_slider, team1_players_slider, team2_players_slider, bat_slider, bowl_slider, wk_slider, ar_slider],
            outputs=[team_output_df, summary_output_json, error_box3]
        )
        fetch_data_btn.click(
            fn=handle_fetch_data,
            inputs=[data_type_radio],
            outputs=[data_output_df]
        )

    # Launch the Gradio app with a public share link
    demo.launch(share=True, debug=True)

else:
    print("\n\nCould not start the application because one or more required files are missing or failed to load.")